In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dutils import get_current_date
import os
from azure.identity.aio import (ClientSecretCredential, DefaultAzureCredential,
                                get_bearer_token_provider)
from autogen_agentchat.agents import UserProxyAgent
from fmputils import *
from yfutils import *
from autogen_core.tools import FunctionTool
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from datetime import date, timedelta, datetime

In [2]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)

True

In [3]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    model="gpt-4o",
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    # azure_ad_token_provider=GetTokenProvider(
    #                 "https://cognitiveservices.azure.com/.default"
    #             ),
    api_key=os.getenv('AZURE_OPENAI_KEY'),
    model_capabilities={
        "vision": False,
        "function_calling": True,
        "json_output": True,
    }
)

In [4]:
get_earning_calls = FunctionTool(fmpUtils.get_earning_calls, description="get a earning call's transcript for a company")

In [5]:
analyst_agent = AssistantAgent(
    name="Market_Analyst",
    system_message="You are an AI Agent. You have knowledge about the management positive and negative outlook, future growths and opportunities based on the earning call transcripts.",
    model_client=model_client,
    tools=[get_earning_calls]
)

In [6]:
team = RoundRobinGroupChat([analyst_agent], max_turns=3)

In [7]:
# company = "Tesla"
stream = team.run_stream(task=f"Summarize and analyze the 2024 (all quarters) earning call transcript for Google and based on management positive and negative outlook, and future growth, predict the stock movement.")
await Console(stream)

---------- user ----------
Summarize and analyze the 2024 (all quarters) earning call transcript for Google and based on management positive and negative outlook, and future growth, predict the stock movement.
FMP api key found successfully.---------- Market_Analyst ----------
[FunctionCall(id='call_9XG5FntSSQX8sUGnlPpCoO9j', arguments='{"ticker_symbol": "GOOGL", "year": "2024"}', name='get_earning_calls')]

---------- Market_Analyst ----------
[FunctionExecutionResult(content='   quarter  year                 date  \\\n0        4  2023  2024-01-30 16:30:00   \n1        3  2023  2023-10-24 16:30:00   \n2        2  2023  2023-07-25 17:00:00   \n3        1  2023  2023-04-25 12:15:00   \n\n                                             content  \n0  Operator: Welcome, everyone. Thank you for sta...  \n1  Operator: Welcome, everyone. Thank you for sta...  \n2  Operator: Welcome, everyone. Thank you for sta...  \n3  Operator: Welcome, everyone. Thank you for sta...  ', call_id='call_9XG5FntSS

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Summarize and analyze the 2024 (all quarters) earning call transcript for Google and based on management positive and negative outlook, and future growth, predict the stock movement.', type='TextMessage'), ToolCallRequestEvent(source='Market_Analyst', models_usage=RequestUsage(prompt_tokens=158, completion_tokens=40), content=[FunctionCall(id='call_9XG5FntSSQX8sUGnlPpCoO9j', arguments='{"ticker_symbol": "GOOGL", "year": "2024"}', name='get_earning_calls')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Market_Analyst', models_usage=None, content=[FunctionExecutionResult(content='   quarter  year                 date  \\\n0        4  2023  2024-01-30 16:30:00   \n1        3  2023  2023-10-24 16:30:00   \n2        2  2023  2023-07-25 17:00:00   \n3        1  2023  2023-04-25 12:15:00   \n\n                                             content  \n0  Operator: Welcome, everyone. Thank you for sta...  

In [16]:
import json 

def summarizeTopic(description: str, topic:str) -> bool:

    AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
    AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

    url = f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{AZURE_OPENAI_DEPLOYMENT_NAME}/chat/completions?api-version={AZURE_OPENAI_API_VERSION}"
    headers = {
        'api-key': os.getenv("AZURE_OPENAI_KEY"),
        "Content-Type": "application/json",
    }

    # Payload for the request
    payload = {
    "messages": [
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": f"You are an AI assistant that will summarize the user input on a {topic}. You will not answer questions or respond to statements that are focused about"
            }
        ]
        }, 
            {
        "role": "user",
        "content": description  
        }
    ],
    "temperature": 0.7,
    "top_p": 0.95,
    "max_tokens": 1200
    }
    # Send request
    response_json = requests.post(url, headers=headers, json=payload)
    print("response_json", response_json.text)
    return json.loads(response_json.text)['choices'][0]['message']['content']


In [17]:
description = """Operator: Good day everyone and welcome to the Apple Inc. Fourth Quarter Fiscal Year 2020 Earnings Conference Call. Today’s call is being recorded. At this time for opening remarks and introductions, I would like to turn things over to Tejas Gala, Senior Analyst, Corporate Finance and Investor Relations. Please go ahead, sir.\nTejas Gala: Thank you. Good afternoon and thank you for joining us. Speaking first today is Apple’s CEO, Tim Cook, and he will be followed by CFO, Luca Maestri. After that, we’ll open the call to questions from analysts. Please note that some of the information you’ll hear during the discussion today, will consist of our forward-looking statements including without limitation those regarding revenue, gross margin, operating expenses, other income and expense, taxes, capital allocation and future business outlook, including the potential impact of COVID-19 on the company’s business, and results of operations. These statements involve risks and uncertainties that may cause actual results or trends to differ materially from our forecast. For more information, please refer to the risk factors discussed in Apple’s most recently filed Annual Report on Form 10-K and the Form 8-K filed with the SEC today, along with the associated press release. Apple assumes no obligation to update any forward-looking statements or information, which speak as of their respective dates. I’d now like to turn the call over to Tim for introductory remarks.\nTim Cook: Thanks Tejas. Good afternoon and thanks for joining the call today. Back in April, I said we were in the most challenging environment in which Apple as a company has ever operated. That atmosphere of uncertainty, of resolve, of making difficult calls with limited information has not only come to define Apple’s year, but each of our lives as individuals across this country, and around the world. It has been a chapter that none of us will forget. In the face of these challenges, Apple stayed relentlessly focused on what we do best, seeing in every obstacle an opportunity to do something new, something creative, something better on behalf of our customers. Today we report a quarter and a fiscal year that reflects that effort. This quarter, Apple achieved revenue of $64.7 billion, a September quarter record, despite the anticipated absence of new iPhone availability during the quarter, and the ongoing impacts of COVID-19, including closures at many of our retail locations. We also set a new all-time record for Mac and Services. Outside of iPhone, each of our product categories saw strong double digit year-over-year growth, despite supply constraints in several product categories."""
topic = "Management Positive Outlook"
print(summarizeTopic(description, topic))

response_json {"choices":[{"content_filter_results":{"hate":{"filtered":false,"severity":"safe"},"protected_material_code":{"filtered":false,"detected":false},"protected_material_text":{"filtered":false,"detected":false},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":false,"severity":"safe"}},"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"In the Apple Inc. Fourth Quarter Fiscal Year 2020 Earnings Conference Call, CEO Tim Cook highlighted the company's adaptability and innovation in response to a challenging environment marked by COVID-19. Despite uncertainties, Apple focused on creativity and improvement, achieving a revenue of $64.7 billion, a September quarter record. The company saw strong double-digit growth in all product categories, except for the anticipated absence of new iPhone availability, and set new records for Mac and Services. This performance underscores Apple's ability to nav